IMPORTING THE LIBRARIES

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.xception import Xception,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
!pip install tensorflow.gpu
!pip install keras


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 578.0 MB 14 kB/s 
     |████████████████████████████████| 438 kB 58.5 MB/s 
     |████████████████████████████████| 1.7 MB 55.6 MB/s 
     |████████████████████████████████| 5.9 MB 62.1 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully u

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


CONFIGURE IMAGE DATA GENERATOR CLASS

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255)

FUNCTIONALIY TO TRAIN AND TEST SET

In [ ]:
training_set=train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                               target_size=(299,299),
                                               batch_size=32,
                                               class_mode='categorical')
test_set=test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                          target_size=(299,299),
                                          batch_size=32,
                                          class_mode='categorical')

Found 3662 images belonging to 5 classes.
Found 734 images belonging to 5 classes.


MODEL BUILDING

In [ ]:
xception = Xception(input_shape=imageSize + [3],weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 3s 0us/step


In [ ]:
#dont train existing weight 
for layer in xception.layers:
  layer.trainable = False

In [ ]:
#our layers-you can add moreif you want
x = Flatten()(xception.output)

ADDING DENSE LAYERS

In [ ]:
prediction = Dense(5, activation='softmax')(x)


In [ ]:
#create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
#view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 configure the learning process

In [ ]:
#tell the model what cost and optimization method to use
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics = ['accuracy']
)



TRAIN THE MODEL

In [ ]:
#fit the model
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=30,
    steps_per_epoch=len(training_set)//32,
    validation_steps=len(test_set)//32
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
3/3 [==============================] - 26s 5s/step - loss: 8.7857 - accuracy: 0.4167
Epoch 2/30
3/3 [==============================] - 14s 5s/step - loss: 8.6083 - accuracy: 0.5000
Epoch 3/30
3/3 [==============================] - 12s 4s/step - loss: 7.2697 - accuracy: 0.5833
Epoch 4/30
3/3 [==============================] - 13s 4s/step - loss: 4.3913 - accuracy: 0.6562
Epoch 5/30
3/3 [==============================] - 13s 4s/step - loss: 8.2424 - accuracy: 0.4479
Epoch 6/30
3/3 [==============================] - 12s 4s/step - loss: 3.6434 - accuracy: 0.6875
Epoch 7/30
3/3 [==============================] - 13s 4s/step - loss: 4.4941 - accuracy: 0.6

SAVE THE MODEL

In [ ]:
model.save('updated-Xception-diabetic-retinopathy.h5')